
# ASMR: App for Safe & Emotion Matter Right

This notebook implements the core functionality of ASMR, focusing on:
1. Speech Emotion Recognition using a 1D-CNN.
2. Nonverbal Sound Recognition using a Capsule Network (CapsNet).

Both models use deep learning to improve accessibility for individuals with hearing impairments.


In [ ]:

# Install necessary libraries (uncomment if needed)
# !pip install tensorflow librosa numpy scikit-learn

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, Sequential
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import os

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Dataset preparation placeholder (to be replaced with actual dataset path)
def load_audio_data(data_dir, sr=16000, duration=3):
    '''
    Load audio data and preprocess it for the models.
    :param data_dir: Path to the directory containing audio files.
    :param sr: Sampling rate.
    :param duration: Duration of audio clips in seconds.
    :return: Preprocessed audio data and labels.
    '''
    audio_data = []
    labels = []
    for label in os.listdir(data_dir):
        class_dir = os.path.join(data_dir, label)
        if os.path.isdir(class_dir):
            for file in os.listdir(class_dir):
                try:
                    audio_path = os.path.join(class_dir, file)
                    y, _ = librosa.load(audio_path, sr=sr, duration=duration)
                    audio_data.append(y)
                    labels.append(label)
                except Exception as e:
                    print(f"Error loading {file}: {e}")
    return np.array(audio_data), np.array(labels)

# Example usage (update 'data_dir' to actual dataset path)
# data_dir = 'path_to_dataset'
# audio_data, labels = load_audio_data(data_dir)


In [ ]:

def build_1d_cnn(input_shape, num_classes):
    '''
    Build a 1D-CNN for speech emotion recognition.
    :param input_shape: Shape of the input data (e.g., (samples, features)).
    :param num_classes: Number of emotion classes.
    :return: Compiled CNN model.
    '''
    model = Sequential([
        layers.Conv1D(64, kernel_size=3, activation='relu', input_shape=input_shape),
        layers.BatchNormalization(),
        layers.MaxPooling1D(pool_size=2),
        layers.Conv1D(128, kernel_size=3, activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling1D(pool_size=2),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Example usage (replace 'input_shape' and 'num_classes' with actual values)
# model = build_1d_cnn(input_shape=(16000, 1), num_classes=7)
# model.summary()


In [ ]:

def build_capsnet(input_shape, num_classes):
    '''
    Build a Capsule Network for nonverbal sound recognition.
    :param input_shape: Shape of the input data (e.g., (samples, features)).
    :param num_classes: Number of sound categories.
    :return: CapsNet model.
    '''
    input_layer = layers.Input(shape=input_shape)
    
    # Convolutional Layer
    conv1 = layers.Conv1D(256, kernel_size=9, strides=1, activation='relu')(input_layer)
    conv2 = layers.Conv1D(256, kernel_size=9, strides=2, activation='relu')(conv1)
    
    # Primary Capsule Layer
    primary_caps = layers.Conv1D(32 * 8, kernel_size=9, strides=2, activation='relu')(conv2)
    primary_caps = layers.Reshape([32, 8])(primary_caps)
    
    # Digit Capsule Layer
    capsule_layer = layers.LSTM(16)(primary_caps)  # Simplified version for demo
    
    # Output Layer
    output = layers.Dense(num_classes, activation='softmax')(capsule_layer)
    
    model = models.Model(inputs=input_layer, outputs=output)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Example usage (replace 'input_shape' and 'num_classes' with actual values)
# model = build_capsnet(input_shape=(16000, 1), num_classes=10)
# model.summary()
